# ACHR_2

In [1]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import yaml
import re

import hypnogram as hp
from ecephys.utils import replace_outliers
import ecephys.plot as eplt
import ecephys.signal.timefrequency as tfr
import ecephys.signal.kd_utils as kd
import ecephys.signal.kd_plotting as kp
import ecephys.signal.kd_pax as kpx
import tdt_xarray as tx

bands_def = dict(delta=(1.5, 4.0), theta=(4.0, 8), sigma=(11,16), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80))

kd_ref = {}
kd_ref['echans'] = [1,2,3]
kd_ref['fchans']=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
kd_ref['analysis_root'] = Path('/Volumes/opto_loc/Data/ACHR_PROJECT_MATERIALS/ACHR_2/ACHR_2-analysis-data')
kd_ref['tank_root'] = Path('/Volumes/opto_loc/Data/ACHR_2_TANK')

# Test-a1

In [2]:
a1p = '/Volumes/opto_loc/Data/ACHR_2_TANK/ACHR_2-test-a1'
a2p = '/Volumes/opto_loc/Data/ACHR_2_TANK/ACHR_2-test-a2'
a3p = '/Volumes/opto_loc/Data/ACHR_2_TANK/ACHR_2-test-a3'
a4p = '/Volumes/opto_loc/Data/ACHR_2_TANK/ACHR_2-test-a4'
a5p = '/Volumes/opto_loc/Data/ACHR_2_TANK/ACHR_2-test-a5'

In [23]:
tt = {}

In [26]:
tt['a1_er'], tt['a1_erspg'] = kd.get_data_spg(a1p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
tt['a1_e'], tt['a1_espg'] = kd.get_data_spg(a1p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
tt['a1_f'], tt['a1_fspg'] = kd.get_data_spg(a1p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

tt['a2_er'], tt['a2_erspg'] = kd.get_data_spg(a2p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
tt['a2_e'], tt['a2_espg'] = kd.get_data_spg(a2p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
tt['a2_f'], tt['a2_fspg'] = kd.get_data_spg(a2p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

tt['a3_er'], tt['a3_erspg'] = kd.get_data_spg(a3p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
tt['a3_e'], tt['a3_espg'] = kd.get_data_spg(a3p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
tt['a3_f'], tt['a3_fspg'] = kd.get_data_spg(a3p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

tt['a4_er'], tt['a4_erspg'] = kd.get_data_spg(a4p, store='EEGr', t1=0, t2=0, channel=kd_ref['echans'])
tt['a4_e'], tt['a4_espg'] = kd.get_data_spg(a4p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
tt['a4_f'], tt['a4_fspg'] = kd.get_data_spg(a4p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

read from t=0s to t=52197.43s
Using 1017.2526 Hz as SEV sampling rate for EEGr
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=52197.43s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=52197.43s
Using 1017.2526 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=21661.76s
Using 1017.2526 Hz as SEV sampling rate for EEGr
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=21661.76s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [82]:
keys = ['a1-Filtered-EEG', 'a3-Filtered-EEG']
test = 'Recording-Config'
def comp_psd(spg1, spg2, median=True, keys=keys, test=test):
    psd1 = spg1.median(dim='time')
    psd2 = spg2.median(dim='time')
    if median is not True: 
        psd1 = spg1.mean(dim='time')
        psd2 = spg2.mean(dim='time')
    psd_df = pd.concat([psd1.to_dataframe('power'), psd2.to_dataframe('power')], keys=keys).rename_axis(index={None: test})
    ax = sns.relplot(data=psd_df, x='frequency', y='power', hue='Recording-Config', col='channel', kind='line')
    x = ax.axes
    x[0,0].set_ylim(0, 250)
    x[0,0].set_xlim(0, 120)
    return ax

In [ ]:

def comp_psd_lfp(spg1, spg2, median=True, keys=keys, test=test, chans=chans):
    psd1 = spg1.sel(channel=chans).median(dim='time')
    psd2 = spg2.sel(channel=chans).median(dim='time')
    if median is not True: 
        psd1 = spg1.sel(channel=chans).mean(dim='time')
        psd2 = spg2.sel(channel=chans).mean(dim='time')
    psd_df = pd.concat([psd1.to_dataframe('power'), psd2.to_dataframe('power')], keys=keys).rename_axis(index={None: test})
    ax = sns.relplot(data=psd_df, x='frequency', y='power', hue='Recording-Config', col='channel', kind='line')
    return ax

In [ ]:

fkeys = ['a2-Filtered-LFP', 'a3-Filtered-LFP']
test = 'Recording-Config'
chans = [3,7,11,15]
plt.close('all')
ax = comp_psd_lfp(tt['a2_fspg'], tt['a3_fspg'], keys=fkeys)

x = ax.axes
#x[0,0].set_ylim(0, 250)
x[0,0].set_xlim(0, 70)

In [30]:
plt.close('all')
band='delta'
chans = [3,8,15]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(tt['a1_espg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=spg.time.max(), title=band.capitalize()+" Power, LFP/EEG-"+str(chan)+", EXPERIMENT")
    #b.set_ylim(0, 300000)

# Exp-1

In [39]:
def stim_durations(onsets, offsets, num):
    durations = list()
    num=num+1
    stims = np.arange(1,num,1)
    for on, off in zip(onsets, offsets):
        dur = off-on
        durations.append(dur)
    return durations
def stim_values(onsets, offsets, num):
    for pulse in zip(onsets, offsets):
        print(pulse)

In [62]:
d = stim_durations(a2x1['stim_on'], a2x1['stim_off'], 11)
d

[7.349411840000002,
 21.18291456,
 29.18399999999997,
 19.456204799999966,
 28.540641280000045,
 56.31496191999997,
 27.641896960000054,
 21.77564671999994,
 22.02136575999998,
 17.584046080000007,
 3706.6069196800004]

In [46]:
stim_values(a2x1['stim_on'], a2x1['stim_off'], 11)

(117.543936, 124.89334784)
(231.6222464, 252.80516096)
(403.63577344000004, 432.81977344)
(492.07574528000004, 511.53195008)
(578.16850432, 606.7091456)
(733.3888000000001, 789.70376192)
(890.42927616, 918.07117312)
(1051.33924352, 1073.11489024)
(1190.7805184000001, 1212.80188416)
(1248.8187904000001, 1266.4028364800001)
(1328.69398528, 5035.3009049600005)


In [220]:
x1p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp1'
#comment

In [221]:
a2x1 = {}
a2x1['e'], a2x1['espg'] = kd.get_data_spg(x1p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x1['f'], a2x1['fspg'] = kd.get_data_spg(x1p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

read from t=0s to t=26071.67s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=26071.67s
Using 1017.2526 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [222]:
a2x1['stim_on'] = tdt.read_block(x1p, evtype=['epocs']).epocs.Bn1_.onset 
a2x1['stim_off'] = tdt.read_block(x1p, evtype=['epocs']).epocs.Bn1_.offset 

read from t=0s to t=26071.67s
read from t=0s to t=26071.67s


In [234]:
a2x1['spikes'] = tdt.read_block(x1p, evtype=['snips']).snips.eSpk

read from t=0s to t=26071.67s


In [233]:
a2x1['spikes']

(4268197, 50)

## Plots

In [20]:
a2x1['f'].sel(channel=2).plot.hist


<xarray.DataArray 'LFP_' (time: 26521344)>
array([-91.65432  , -52.163403 ,  13.439454 , ...,  34.76087  ,
        -7.074648 ,   0.2540025], dtype=float32)
Coordinates:
  * time       (time) float64 0.0 0.000983 0.001966 ... 2.607e+04 2.607e+04
    channel    int64 2
    timedelta  (time) timedelta64[ns] 00:00:00 ... 07:14:31.540488775
    datetime   (time) datetime64[ns] 2021-10-18T10:46:59.999999 ... 2021-10-1...
Attributes:
    units:    uV
    fs:       1017.252625

In [61]:
band='delta'
for chan in [1,2,3]:
    b, s = kp.plot_spectrogram_with_bandpower(a2x1['espg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=26000, title=band.capitalize()+" Power, EEG-"+str(chan)+", ACHR_2 1-hr Pulse")
    b.set_ylim(0, 4500)
    b.axvspan(
            a2x1["stim_on"][10],
            a2x1["stim_off"][10],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
band='delta'
for chan in [1,3]:
    b, s = kp.plot_spectrogram_with_bandpower(a2x1['fspg'], bands_def, band, hyp=None, channel=chan, start_time=0, end_time=26000, title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2 1-hr Pulse")
    #b.set_ylim(0, 200000)
    b.axvspan(
            a2x1["stim_on"][10],
            a2x1["stim_off"][10],
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exp-2

## Load data

In [2]:
x2p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp2'
b2p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp2-bl'
p2p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp2-post'
a2x2 = {}
a2x2['e'], a2x2['espg'] = kd.get_data_spg(x2p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x2['f'], a2x2['fspg'] = kd.get_data_spg(x2p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])
a2x2['be'], a2x2['bespg'] = kd.get_data_spg(b2p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x2['bf'], a2x2['bfspg'] = kd.get_data_spg(b2p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

In [5]:
a2x2['pe'], a2x2['pespg'] = kd.get_data_spg(p2p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x2['pf'], a2x2['pfspg'] = kd.get_data_spg(p2p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

read from t=0s to t=85286.54s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=85286.54s
Using 1017.2526 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [6]:
a2x2['stim_on'] = tdt.read_block(x2p, evtype=['epocs']).epocs.Bn1_.onset 
a2x2['stim_off'] = tdt.read_block(x2p, evtype=['epocs']).epocs.Bn1_.offset 

read from t=0s to t=102476.04s
read from t=0s to t=102476.04s


## Exp-2 PLOTS

In [64]:
plt.close('all')
band='delta'
spg = a2x2['bfspg']
chans = [2,4,6,8,10,12,14,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=spg.time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-2 Baseline")
    b.set_ylim(0, 200000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [249]:
plt.close('all')
band='sigma'
spg = a2x2['bfspg']
chans = [12,14]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=spg.time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-2 Baseline")
    b.set_ylim(0, 10000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
plt.close('all')
band='delta'
spg = a2x2['fspg']
chans = [2,4,6,8,10,12,14,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=spg.time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-2")
    b.set_ylim(0, 200000)
    b.axvspan(
            2803.79,
            16871.40,
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='darkslategrey',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [258]:
plt.close('all')
band='sigma'
spg = a2x2['fspg']
chans = [12,14]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=spg.time.max(), title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-2")
    b.set_ylim(0, 10000)
    b.axvspan(
            2803.79,
            16871.40,
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Exp-3

In [44]:
def plot_bp_set(spg, bands, hyp, channel, start_time, end_time, ss=12, figsize=(15,5), title=None):
    spg = spg.sel(channel=channel, time=slice(start_time, end_time))
    bp_set = kd.get_bp_set2(spg, bands)
    bp_set = kd.get_smoothed_ds(bp_set, smoothing_sigma=ss)
    ax_index = np.arange(0, len(bands))
    keys = kd.get_key_list(bands)

    fig, axes = plt.subplots(ncols=1, nrows=len(bands), figsize=figsize)

    for i, k in zip(ax_index, keys):
        fr = bp_set[k].f_range
        fr_str = '('+str(fr[0]) + ' -> ' +str(fr[1])+' Hz)'
        ax = sns.lineplot(x=bp_set[k].time, y=bp_set[k], ax=axes[i])
        ax.set_ylabel('Raw '+k.capitalize()+' Power')
        ax.set_title(k.capitalize()+' Bandpower '+fr_str)
    fig.suptitle(title)
    fig.tight_layout(pad=1)
    return fig, axes

In [2]:
x3p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp3'
b3p = '/Volumes/opto_loc/Data/ACHR_2/ACHR_2_TANK/ACHR_2-exp3-bl'

In [3]:
a2x3 = {}
a2x3['e'], a2x3['espg'] = kd.get_data_spg(x3p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x3['f'], a2x3['fspg'] = kd.get_data_spg(x3p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

read from t=0s to t=191466.87s
Using 1017.2526 Hz as SEV sampling rate for EEG_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset
read from t=0s to t=191466.87s
Using 1017.2526 Hz as SEV sampling rate for LFP_
Remember to save all data in xset-style dictionary, and to add experiment name key (key = "name") before using save_xset


In [4]:
a2x3['be'], a2x3['bespg'] = kd.get_data_spg(b2p, store='EEG_', t1=0, t2=0, channel=kd_ref['echans'])
a2x3['bf'], a2x3['bfspg'] = kd.get_data_spg(b2p, store='LFP_', t1=0, t2=0, channel=kd_ref['fchans'])

a2x3['stim_on'] = tdt.read_block(x3p, evtype=['epocs']).epocs.Bn1_.onset 
a2x3['stim_off'] = tdt.read_block(x3p, evtype=['epocs']).epocs.Bn1_.offset 

NameError: name 'b2p' is not defined

In [52]:
a2x3['stim_on']

array([2355.13352192])

In [53]:
a2x3['stim_off']

array([20421.34355968])

In [58]:
plt.close('all')
band='delta'
spg = a2x3['espg']
chans = [1,2,3]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=43200, title=band.capitalize()+" Power, EEG-"+str(chan)+", ACHR_2, Experiment-3")
    #b.set_ylim(0, 10000)
    b.axvspan(
            2355.13,
            20421.34,
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
plt.close('all')
band='delta'
spg = a2x3['fspg']
chans = [10,12,14,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=55000, title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-3")
    #b.set_ylim(0, 10000)
    b.axvspan(
            2355.13,
            20421.34,
            alpha=0.3,
            color='lightskyblue',
            zorder=1000,
            ec="none",
        )
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
band='delta'
spg = a2x3['bfspg']
chans = [10,12,14,16]
for chan in chans:
    b, s = kp.plot_spectrogram_with_bandpower(spg, bands_def, band, hyp=None, channel=chan, start_time=0, end_time=43200, title=band.capitalize()+" Power, LFP-"+str(chan)+", ACHR_2, Experiment-3 Baseline")
    #b.set_ylim(0, 10000)
    b.axvspan(
            43200,
            86400,
            alpha=0.3,
            color='midnightblue',
            zorder=1000,
            ec="none",
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
bands2 = dict(delta=(0.5, 4), theta=(4, 8), sigma=(11,16), beta = (13, 20), low_gamma = (40, 55), high_gamma = (65, 80), zeta=(130,300))
f, a = plot_bp_set(tspg, bands=bands2, hyp=None, channel=1, start_time=0, end_time=tspg.time.max(), ss=2, figsize=(20,18), title='')
a[6].set_ylim(0,1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 1000.0)